In [ ]:
"""
@Description :   
@Author      :   Xubo Luo 
@Time        :   2024/06/19 21:18:51
"""

import torch
from torch.autograd import Variable
from utils import subsequent_mask
from setting import MAX_LENGTH, DEVICE
import numpy as np
from utils import *

In [ ]:
input_sentence = "I am a student"
output = translate_large(input_sentence, 'en', 'zh')
print(output)

In [ ]:
import torch.nn.functional as F
import math
import torch
from nltk.tokenize import word_tokenize
from torch import nn
from torch.utils.data import Dataset, DataLoader
from torch.autograd import Variable
import copy
import numpy as np
import os
import re
import sacrebleu
import random
import time
import jieba
from nltk.tokenize.treebank import TreebankWordDetokenizer

def is_english_sentence(sentence):
    # 使用正则表达式检查句子中是否包含英文字母
    english_pattern = re.compile(r'[a-zA-Z]')
    match = english_pattern.search(sentence)
    # True 表示这是英文句子
    if match: 
        return True
    else:
        return False
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.bleu_score import SmoothingFunction

# 这个smooth防止句子长度小于4而出现报错
smooth = SmoothingFunction().method1
def compute_bleu4(tokenizer, random_integers, model, device):
    """
    计算BLEU4
    :param tokenizer: tokenizer
    :param random_integers: 这个是随机选择的测试集数据的编号
    :param model: 模型
    :param device: 设备
    :return:
    """
    # m1,m2存放英文的原数据与模型输出数据
    m1, m2 = [], []
    # m3,m4存放英文的原数据与模型输出数据
    m3, m4 = [], []
    model.eval()
    # 存放测试数据
    da = []
    # 将随机选择的测试集数据编号添加到da中
    for i in random_integers:
        da.append(tokenizer.test[i])
    # 对da中的数据进行编码
    labels, x, _ = tokenizer.encode_all(da)
    with torch.no_grad():
        # 预测
        y = predict(x, model, tokenizer, device)
    # 这个p用于记录y的索引
    p = 0
    # 用于保存有效的索引
    itg = []
    # 这里我限制输入数据全部有效，如果有无效的数据，直接放弃本次计算
    if len(y) != 10:
        return 0
    for i in labels:
        # 取出有效的索引
        itg.append(random_integers[i])
    # 将真实数据与预测数据分别放到m1,m2,m3,m4中
    for i in itg:
        if is_english_sentence(tokenizer.test[i][1]):
            m1.append(tokenizer.test[i][1])
            m2.append([y[p]])
        else:
            m3.append(list(jieba.cut(tokenizer.test[i][1])))
            m4.append([list(jieba.cut(y[p]))])
        p += 1
    smooth = SmoothingFunction().method1
    # 计算英文的bleu4
    b1 = [sacrebleu.sentence_bleu(candidate, refs).score for candidate, refs in zip(m1, m2)]
    # 计算中文的bleu4
    for i in range(len(m4)):
        b2 = sentence_bleu(m4[i], m3[i], weights=(0.25, 0.25, 0.25, 0.25), smoothing_function=smooth) * 100
        b1.append(b2)
#     print(b1)
#     print(sum(b1)/len(b1))
    return sum(b1)/len(b1)

In [13]:
# cacluate bleu score
from nltk.translate.bleu_score import sentence_bleu
reference = [['我', '是', '一个', '学生']]
candidate = ["我", '是',  "一个", '学生']
# candidate = ['I', 'am', 'a', 'student']

score = sentence_bleu(reference, candidate, weights=(0.25, 0.25, 0.25, 0.25))
print(score)


1.0
